In [ ]:
###### import os
import PyPDF2

# Input paths
folder_path = "case_studies"
output_txt_file = "case_studies.txt"

# Initialize an empty string to accumulate text from all PDFs
all_text = ""

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        
        # Open and read the PDF file
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            for page_num in range(len(reader.pages)):
                all_text += reader.pages[page_num].extract_text()

# Write the combined text into a single file
with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
    txt_file.write(all_text)

print(f"Text from all PDFs in {folder_path} has been written to {output_txt_file}.")



In [1]:
import os
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from apikey import apikey
from langchain import OpenAI

os.environ['OPENAI_API_KEY'] = apikey

# The LangChain component we'll use to get the documents
from langchain.chains import Retrieval


#Run from Here for case analysis
#loader = TextLoader('case_studies.txt')
#pages = loader.load_and_split()
loader = TextLoader('case_studies.txt')
doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3900, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

fs = LocalFileStore("./cached_embeddings")
underlying_embeddings = OpenAIEmbeddings()
embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, fs, namespace=underlying_embeddings.model
)

db = FAISS.from_documents(docs, embedder)


/home/prash/.local/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [6]:
db.save_local("faiss_index_cases")

In [4]:


llm = OpenAI(temperature=0.6, openai_api_key=apikey)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=db.as_retriever())

In [5]:
query = "what is the law of variable proportions?"
qa.run(query)

' \n\nThe law of variable proportions states that when one of the factors of production is held fixed in supply, successive additions of the other factors will lead to an increase in returns up to a point, but beyond this point returns will diminish. This law is closely related to the concept of elasticity, which measures the responsiveness of the quantity demanded of a good to its price. Price elasticity of demand is measured as the percentage change in quantity demanded that occurs in response to a percentage change in price. Cross-elasticity of demand measures the responsiveness of the quantity demanded of one good to a price change in another good. The law of variable proportions can be used to understand the responsiveness of demand to changes in price or other factors, and it can also be applied to market sizing exercises to help a business understand the relationship between factors of production and how these factors can affect returns. For example, in the case of a glass bottl

In [8]:
new_db = FAISS.load_local("faiss_index_cases", embedder)

In [10]:
llm = OpenAI(temperature=0.6, openai_api_key=apikey)
qa2 = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=new_db.as_retriever())

In [14]:
query2 = "what questions should you ask for a case similar to hoopla?"
qa2.run(query2)

"\n\n· Has the customer base shifted in any way? \n· Are there any external factors that could be affecting customer behavior (e.g. economic climate, new competition, healthcare reform)? \n· Are there any internal factors that could be affecting customer behaviour (e.g. changes in service, quality, pricing, product features, promotion, distribution channels)? \n· Are there any changes in customer preferences or buying habits? \n· Are there any new marketing or advertising strategies that have been implemented? \n· Has there been any changes in the company's supply chain or distribution methods? \n· Are there any changes in the company's target market or customer demographics? \n· Are there any changes in the company's customer service or customer experience?\n· Have costs increased?  \n· Have revenues declined?  \n· Have prices been changed?  \n· Have new video stores opened in the area?  \n· Are fewer customers coming to the store?  \n· Are customers renting fewer videos?  \n· Have ot